In [ ]:
# | default_exp read_soil_file

In [4]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [5]:
# | export
from pathlib import Path
import os
import pandas as pd
from typing import Dict
import pandera as pa
from pandera.typing import DataFrame, Series
from sureau_ecos_py.create_modeling_options import create_modeling_options
import numpy as np



In [213]:
# | export
# This class is created for validating the input  dataframe
# If the data don't follow the structure in the class the function will fail
class SoilFile(pa.SchemaModel):
    """Schema for validating the input soil parameter file """

    Name: Series[str] = pa.Field(
        description= "Parameter names"
    )
    Value: Series[float] = pa.Field(
        description= "Parameter values"
    )

@pa.check_types(lazy=True)
def read_soil_file(file_path:Path, # Path to a csv file containing parameter values i.e path/to/file_name.csv
                   modeling_options:Dict = None, # Dictionary created using the `create_modeling_options` function
                   sep:str = ';' # CSV file separator can be ',' or ';'
                   )-> DataFrame[SoilFile]:
    """
    Function for reading a data frame containing information about soil
    characteristics
    """
    # Make sure that modeling_options is a dictionary ---------------------------
    assert isinstance(
        modeling_options, Dict
    ), f"modeling_options must be a dictionary not a {type(modeling_options)}"

    # Read data frame -----------------------------------------------------------
    if os.path.exists(file_path):
        try:

            # Read file
            soil_data = pd.read_csv(file_path,header = 0, sep = sep)

        # Raise error if soil data don't follow the SoilFile Schema
        except pa.errors.SchemaErrors as err:
            print(err)
    else:
        print(f"file: {file_path}, does not exist, check presence or spelling")

    # Setting common parameters for WB_soil (regardless of the options) ---------
    if modeling_options['pedo_transfer_formulation'] == 'vg':

        # 14 params
        params = np.array(["rfc_1", "rfc_2", "rfc_3", "depth_1", "depth_2",
                           "depth_3", "wilting_point", "alpha_vg", "n_vg",
                           "i_vg", "ksat_vg", "saturation_capacity_vg",
                           "residual_capacity_vg","g_soil_0"], dtype=object)


    if modeling_options['pedo_transfer_formulation'] == 'campbell':

        # 12 params
        params = np.array(["rfc_1", "rfc_2", "rfc_3", "depth_1", "depth_2",
                           "depth_3", "wilting_point","ksat_campbell",
                           "saturation_capacity_campbell", "b_camp","psie",
                           "g_soil_0"], dtype=object)


    # Filter params inside the read data set ------------------------------------
    soil_data = soil_data[soil_data['Name'].isin(params)]

    # Make sure that no parameters are missing  ---------------------------------
    # (12 or 14)
    for each_parameter in params:

        # Raise error if a parameter is missing
        if each_parameter not in np.array(soil_data["Name"]):
            raise ValueError(f'{each_parameter} not provided in input soil parameter file, check presence or spelling\n')

    # Make sure there are no duplicate parameters -------------------------------
    if len(soil_data['Name']) is not len(set(soil_data['Name'])):
        raise ValueError('Parameter repeated several times in input soil parameter file')

    # return dict

    return soil_data

In [163]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="turgor",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="vg",
)
modeling_options_dict

defaultdict(list,
            {'constant_climate': False,
             'etp_formulation': 'pt',
             'rn_formulation': 'linear',
             'pedo_transfer_formulation': 'vg',
             'time_step_for_evapo': 2,
             'time': array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22]),
             'reset_swc': True,
             'avoid_water_soil_transfer': True,
             'comp_options': defaultdict(list,
                         {'numerical_scheme': 'implicit',
                          'nsmalltimesteps': 12.0,
                          'lsym': 1,
                          'ssym': 1,
                          'clapo': 1,
                          'ctapo': 1,
                          'eord': 1,
                          'lcav': 1,
                          'scav': 1}),
             'stomatal_reg_formulation': 'turgor',
             'soil_evapo': True,
             'defoliation': True,
             'threshold_mortality': 51,
             'transpiration_model': 'jarv

In [212]:

read_soil_file("./test_folder/soil_example.csv",
               modeling_options=modeling_options_dict,
               sep=';')

ValueError: ksat_vg not provided in input soil parameter file, check presence or spelling


,Name,Value
0,rfc_1,75.00000
1,rfc_2,82.00000
2,rfc_3,94.00000
3,field_capacity,0.38000
4,wilting_point,0.12379
5,alpha_vg,0.00050
6,n_vg,1.55000
7,i_vg,0.50000
8,ksat_vg,1.69000
9,saturation_capacity_vg,0.50000


In [175]:
test_data = pd.read_csv("./test_folder/soil_example.csv", sep=';')
#test_data[params.isin(test_data.Name)]

print(set(test_data['Name']))
#print(set(params))


{'wilting_point', 'saturation_capacity_vg', 'depth_3', 'ksat_campbell', 'n_vg', 'offset_psoil', 'i_vg', 'rfc_3', 'residual_capacity_vg', 'depth_1', 'field_capacity', 'saturation_capacity_campbell', 'depth_2', 'b_camp', 'g_soil_0', 'rfc_2', 'psie', 'alpha_vg', 'ksat_vg', 'rfc_1'}


,Name,Value
0,rfc_1,75.00000
1,rfc_2,82.00000
2,rfc_3,94.00000
4,wilting_point,0.12379
11,g_soil_0,30.00000
12,depth_1,0.20000
13,depth_2,1.00000
14,depth_3,4.00000
16,b_camp,-4.00000
17,psie,-0.02500
